In [21]:
# imports
import pandas as pd
import numpy as np
import random
import statistics as stat

traindf = pd.read_pickle("./pickles/df_small_train.pkl")

# display(traindf[traindf["position"] == 11])

In [22]:
def random_solution(df):
    
    # csv_name = "./results/" + csv_name
    
    # get list of hotels per search id
    results_per_search = df.groupby('srch_id').size()
    
    column_values = df["srch_id"].values
    
    ids =  pd.unique(column_values) 
    
    all_ranks = []
    
    for i, ideetje in enumerate(ids):
        df_temp = df[df["srch_id"] == ideetje]
        
        ranks = df_temp["position"].values
        
        # shuffle ranks
        random.shuffle(ranks)
        all_ranks.extend(ranks)

    df["predicted_rank"] = all_ranks
    return df

In [30]:
def indiv_search_ndcg_score(true_rank, predicted_rank, booked, clicked):
    # holder for ideal score
    idcg = 0
    
    # holder for our score
    dcg = 0

    # itereer over elk resultaat in de search
    for i in range(len(true_rank)):
        # hotels die geboekt zijn zijn heel belangrijk (5)
        if booked[i] == 1:
            
            # increment ideal score
            if i != 0:
                idcg = idcg + (5 / np.log2(i+1))
            else:
                idcg += 5
            
            # if predicted well, increment score
            if true_rank[i] == predicted_rank[i]:
                if i != 0:
                    dcg = dcg + (5 / np.log2(i+1))
                else:
                    dcg += 5
                
        # hotels waarop geklikt is zijn een beetje belangrijk (1)
        elif clicked[i] == 1:
            
            # increment ideal score
            if i != 0:
                idcg = idcg + (1 / np.log2(i+1))
            else:
                idcg += 1
            
            # if predicted well, increment score
            if true_rank[i] == predicted_rank[i]:
                dcg = dcg + (1 / np.log2(i+2))
    
    return dcg/idcg

In [31]:
# score function for every search (thus iterate over groupby object)
def dataframe_ndcg_score(solution):
    all_scores = []
    for srchid, group in solution.groupby('srch_id'):

        # true_rank
#         true_rank = np.arange(1, len(group) + 1)
        true_rank = np.asarray(group['position'])

        # predicted_rank
        predicted_rank = np.asarray(group['predicted_rank'])

        # booking_bool
        booked = np.asarray(group['booking_bool'])

        # click_bool
        clicked = np.asarray(group['click_bool'])

        all_scores.append(indiv_search_ndcg_score(true_rank, predicted_rank, booked, clicked))
    return np.nanmean(all_scores)

In [32]:
for i in range(100):
    print(i, dataframe_ndcg_score(random_solution(traindf)))

0 0.060200939918540294
1 0.050062934389496286
2 0.05449232768506828
3 0.04468771659977545
4 0.055875426109889985
5 0.04686368146373029
6 0.048232894760489184
7 0.05034767035075709
8 0.04969006075166653
9 0.05608454209264268
10 0.053993183020530744
11 0.05131616973945564
12 0.05033603682527593
13 0.05331115289890637
14 0.062100347084515485
15 0.04534091562019292
16 0.05832765039940446
17 0.055354179713086385
18 0.054792496055324914
19 0.06270004977840951
20 0.05300528937206096
21 0.05798855448802609
22 0.0496431569589935
23 0.06401871883179604
24 0.04890728737240452
25 0.05097675584846933
26 0.051421272555722895
27 0.058774230071276114
28 0.053259799439209686
29 0.054275790950331006
30 0.05600482890489653
31 0.052793240953972635
32 0.050143140298864984
33 0.05218263993012246
34 0.04556090951960028
35 0.05567528251947118
36 0.046300370522046275
37 0.05673567069817228
38 0.053775795719488345
39 0.05258100693201034
40 0.04409457544493886
41 0.04790976544244186
42 0.04591897286713665
43 0.0